# Silhouette

This notebook shows how to extract the silhouette of a person from a video by using detectron2.

In [ ]:
!pip install --upgrade 'git+https://github.com/facebookresearch/detectron2.git@v0.6'

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband05'
scenario = '101_natural_lighting'

In [ ]:
frames, meta = dataset.get_video_rgb(subject, scenario, 1, 2000)

In [ ]:
# Show the first frame
import matplotlib.pyplot as plt

plt.imshow(frames[0])
plt.show()

In [ ]:
im = frames[0]

In [ ]:
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

In [ ]:
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

plt.imshow(out.get_image()[:, :, ::-1])
plt.show()